In [1]:
import numpy as np
#np.set_printoptions(precision=5, linewidth=200, suppress=True)
import psi4
from helper_PFCI import PFHamiltonianGenerator

In [2]:
mol_str = """
O
H 1 1.1
H 1 1.1 2 104
symmetry c1
"""

options_dict = {'basis': 'sto-3g',
                  'scf_type': 'pk',
                  'e_convergence': 1e-10,
                  'd_convergence': 1e-10
                  }

mol = psi4.geometry(mol_str)


psi4.set_options(options_dict)


In [6]:
# photon energy
#omega_val = 2.0 / psi4.constants.Hartree_energy_in_eV
omega_val = 2.0

print(omega_val)

# lambda vector
lambda_vector = np.array([.0, .0, .01])
#lambda_vector = np.array([0, 0, 0.0])

# Build PF Hamiltonian matrix 
H2_PF = PFHamiltonianGenerator(1, mol_str, options_dict, lambda_vector,omega_val, ignore_coupling=False)

# uncomment to print
#print(H2_PF.H_PF)


2.0

Scratch directory: /tmp/

*** tstart() called on CHEM353Q05NALT
*** at Wed Mar 15 15:06:17 2023

   => Loading Basis Set <=

    Name: STO-3G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line    81 file /Users/jfoley19/Code/psi4/objdir-Release/stage/share/psi4/basis/sto-3g.gbs 
    atoms 2-3 entry H          line    19 file /Users/jfoley19/Code/psi4/objdir-Release/stage/share/psi4/basis/sto-3g.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c1
    Full point group: C2v

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center              X                  Y      

Properties computed using the SCF density matrix


 Multipole Moments:

 ------------------------------------------------------------------------------------
     Multipole            Electronic (a.u.)      Nuclear  (a.u.)        Total (a.u.)
 ------------------------------------------------------------------------------------

 L = 1.  Multiply by 2.5417464519 to convert [e a0] to [Debye]
 Dipole X            :          0.0000000            0.0000000            0.0000000
 Dipole Y            :          0.0000000            0.0000000            0.0000000
 Dipole Z            :         -0.5237699            1.1272912            0.6035213
 Magnitude           :                                                    0.6035213

 ------------------------------------------------------------------------------------

Start SCF iterations:

Canonical RHF One-electron energy = -120.1995577635351395
CQED-RHF One-electron energy      = -120.1992268786213742
Nuclear repulsion energy          = 8.002366

/Users/jfoley19/Code/qed-ci/src/helper_PFCI.py:929: ComplexWarning: Casting complex values to real discards the imaginary part
  _D1_RR[_i, _i] = _D0[0, 0] + _D1[0, 0]
/Users/jfoley19/Code/qed-ci/src/helper_PFCI.py:943: ComplexWarning: Casting complex values to real discards the imaginary part
  _D1_RS[_i, _a] = _D0[0, _ket_I] + _D1[0, _ket_I]
/Users/jfoley19/Code/qed-ci/src/helper_PFCI.py:958: ComplexWarning: Casting complex values to real discards the imaginary part
  _D1_SS[_a, _b] += (_D0[_ket_I, _ket_J] + _D1[_ket_I, _ket_J]) * (
/Users/jfoley19/Code/qed-ci/src/helper_PFCI.py:961: ComplexWarning: Casting complex values to real discards the imaginary part
  _D1_SS[_i, _j] += (_D0[_ket_I, _ket_J] + _D1[_ket_I, _ket_J]) * (


In [ ]:
# diagonalize the Hamiltonian matrix
e_fci, wavefunctions = np.linalg.eigh(H2_PF.H_PF)

#print the ground-state and first excited state
#print("Psi4 |G> is ",fci_e)
print("|G> is ",e_fci[0])
print("|LP> is ",e_fci[1])
#|G> is  -75.01298018271805
#|LP> is  -75.01298018271805
assert np.isclose(e_fci[0],-74.94207989868084)

ndocc = 5
n_act_el = 4
n_act_orb = 4

n_in_orb=ndocc-n_act_el//2
n_ac_el_half=n_act_el//2
print(n_ac_el_half)
inactive_list=list(x for x in range(n_in_orb))
print(inactive_list)
print('Generating all determinants in active space')
t = time.time()
singlesDets = []
for alpha in combinations(range(n_act_orb), n_ac_el_half):
    alphalist=list(alpha)
    alphalist=[x+n_in_orb for x in alphalist]
    alphalist[0:0]=inactive_list
    alpha=tuple(alphalist)
    #alpha_ex_level = compute_excitation_level(alpha, ndocc)
    for beta in combinations(range(n_act_orb), n_ac_el_half):
        betalist=list(beta)
        betalist=[x+n_in_orb for x in betalist]
        betalist[0:0]=inactive_list
        beta=tuple(betalist)
        #beta_ex_level = compute_excitation_level(beta, ndocc)
        #if alpha_ex_level + beta_ex_level == 1:
        print(F' adding alpha: {alpha} and beta: {beta}\n') 
        singlesDets.append(Determinant(alphaObtList=alpha, betaObtList=beta))